In [9]:
import jax
import jax.numpy as jnp
from jax import jit, vmap, grad
from jax import random
import optax
import numpy as np
import pickle

alpha=2.7
beta=3.9

In [10]:
def init_network_params(sizes, key=random.PRNGKey(4)):
    def random_layer_params(m, n, key, scale=1e-2):
        w_key, b_key = random.split(key)
        return scale * random.uniform(w_key, (n, m), minval=1, maxval=3), scale * random.uniform(b_key, (n,), minval=1, maxval=3)

    keys = random.split(key, len(sizes))
    return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

def predict(NN, x):
    activations = x.reshape((-1, ))
    for w, b in NN[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = jax.nn.sigmoid(outputs)
    final_w, final_b = NN[-1]
    logit = jnp.dot(final_w, activations) + final_b
    logit = 1/(1+jnp.exp(-logit))
    return logit.squeeze()

vpredict = vmap(predict, in_axes=(None, 0))
gpredict = grad(predict, argnums=1)
vgpredict = vmap(gpredict, in_axes=(None, 0))

In [11]:
def pi(x):
    from scipy.special import gamma
    B_ab = (gamma(alpha) * gamma(beta)) / gamma(alpha + beta)
    pdf = (x**(alpha - 1)) * ((1 - x)**(beta - 1)) / B_ab
    return pdf
vpi = vmap(pi, in_axes=(0,))

def pi_pre(NN, x):
    result = gpredict(NN, x)
    return result

@jit
def obj(NN, x_batch):
    def obj1(NN, x):
        return pi(x) - pi_pre(NN, x)
    vobj1 = vmap(obj1, in_axes=(None, 0))

    result = vobj1(NN, x_batch)
    result = jnp.square(result)
    result = jnp.mean(result)
    return result
jgo = jit(grad(obj))

@jit
def evaluation(NN, test_zu):
    z_batch, u_batch = test_zu[:, 0], test_zu[:, 1]
    result = jnp.square(vpredict(NN, z_batch) - u_batch)
    result = jnp.mean(result)
    return result

In [13]:
LAYER_SIZES = [1, 10, 10, 10, 1]
NN = init_network_params(LAYER_SIZES, random.PRNGKey(15))

optimizer= optax.adam(0.001)
opt_state = optimizer.init(NN)
test_zu = np.load(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Beta checkpoints (PINN)/test_zu(a{alpha}b{beta}).npy')
lowest = evaluation(NN, test_zu)
Lloss = []
Llowest = []
NN_best = NN
for epoch in range(2000000):
    x_batch = np.random.uniform(0, 1, (10000, 1))

    grads = grad(obj)(NN, x_batch)
    updates, opt_state = optimizer.update(grads, opt_state)
    NN = optax.apply_updates(NN, updates)

    Lloss.append(obj(NN, x_batch).item())
    Llowest.append(lowest.item())
    print(f"Epoch: {epoch}, loss:{obj(NN, x_batch)}, {lowest}")

    if  evaluation(NN, test_zu)<lowest:
        lowest = evaluation(NN, test_zu)
        NN_best = NN

    if epoch % 10000 == 0:
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Beta checkpoints (PINN)/a{alpha}b{beta}/Lloss.npy', Lloss)
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Beta checkpoints (PINN)/a{alpha}b{beta}/Llowest.npy', Llowest)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Beta checkpoints (PINN)/a{alpha}b{beta}/NN (best).pkl', 'wb') as f:
            pickle.dump(NN_best, f)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Beta checkpoints (PINN)/a{alpha}b{beta}/NN_{epoch}.pkl', 'wb') as f:
            pickle.dump(NN, f)

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch: 1995000, loss:1.448594417752247e-07, 1.7897418269630094e-10
Epoch: 1995001, loss:1.4441920370700245e-07, 1.7897418269630094e-10
Epoch: 1995002, loss:1.4600584563595476e-07, 1.7897418269630094e-10
Epoch: 1995003, loss:1.4607844889269472e-07, 1.7897418269630094e-10
Epoch: 1995004, loss:1.373198017518007e-07, 1.7897418269630094e-10
Epoch: 1995005, loss:1.4100420742124697e-07, 1.7897418269630094e-10
Epoch: 1995006, loss:1.4671906001240131e-07, 1.7897418269630094e-10
Epoch: 1995007, loss:1.545364227695245e-07, 1.7897418269630094e-10
Epoch: 1995008, loss:1.566792633411751e-07, 1.7897418269630094e-10
Epoch: 1995009, loss:1.3944438137514226e-07, 1.7897418269630094e-10
Epoch: 1995010, loss:1.5816283394087804e-07, 1.7897418269630094e-10
Epoch: 1995011, loss:1.359824892688266e-07, 1.7897418269630094e-10
Epoch: 1995012, loss:1.5627686877905944e-07, 1.7897418269630094e-10
Epoch: 1995013, loss:1.4173690487950807e-07, 1.7897418269630094e-10
Epoch: 1995014, loss:1.372